## Example file of notebook of translating SQL query into MongoDB query
#### Eddy Liu, Qixiang Jiang
#### DS4300

## First we install the package for connecting mongoDB

In [1]:
# pip install pymongo

In [4]:
from pymongo import MongoClient
import numpy as np
import pandas as pd

## # Define keywords and comparison operators

In [5]:
keywords = ["select", "from", "where"]
comp_ops = {
    "=": "$eq",
    ">": "$gt",
    "<": "$lt",
    ">=": "$gte",
    "<=": "$lte",
    "or": "$or",
    "and": "$and"
}

# Create DataFrame to store the comparison operators mappings
df_comparison_ops = pd.DataFrame(comp_ops.items(), columns=["SQL", "MQL"])
df_comparison_ops

,SQL,MQL
0,=,$eq
1,>,$gt
2,<,$lt
3,>=,$gte
4,<=,$lte
5,or,$or
6,and,$and


In [43]:
def query_comparison_ops(string,from_ql,to_ql):
    return df_comparison_ops[df_comparison_ops[f"{from_ql}"]==f"{string}"][f"{to_ql}"].item()

assert query_comparison_ops(">","SQL","MQL") == "$gt"
assert query_comparison_ops("<","SQL","MQL") == "$lt"
assert query_comparison_ops("$gte","MQL","SQL") == ">="

In [47]:
# Sample SQL-like query
# query = """SELECT (a, SUM(b)) from (  \n select (*) from   (x) )  WHERE (a > 30 OR   b = "hello world"   )"""
query = """SELECT (sum(quantity)) from (Test_data)"""
# Clean up the query string
query = " ".join(query.lower().split())

query

'select (sum(quantity)) from (test_data)'

In [50]:
def parse_paren(string):
    """
    runtime: O(n)
    Parses parenthesis character by character.
    
    returns a tuple of "outer" parentheses.
    """
    lvl = 0
    last_open = None
    parens_dict = {}
    for i,char in enumerate(string):
        assert lvl >= 0, "check parenthesis"
        if char == "(":
            if lvl==0:
                last_open = i
            lvl+=1
        elif char == ")":
            lvl-=1
            if lvl==0:
                parens_dict[(last_open,i+1)] = string[last_open+1:i]
    
    return parens_dict

# parse_paren(query)

# Parse parentheses in the query string
paren_tuples = parse_paren(query)

paren_tuples = parse_paren(query)
for i in range(len(paren_tuples.keys())):
    i = (-i-1)
    vals = list(paren_tuples.keys())[i]
    query = (query[:vals[0]]+"{}"+query[vals[1]:])

query

'select {} from {}'

In [51]:
# Parse the cleaned-up query string into a dictionary
q_dict = {}
last_kw = False
appendable = ""
paren_tuples_i = 0

q_list = query.split()

for i, element in enumerate(q_list):
    if element in keywords:
        if i == 0:
            last_kw = element
        else:
            q_dict[last_kw] = appendable
            last_kw = element
            appendable = ""
    elif element == "{}":
        appendable += paren_tuples[list(paren_tuples.keys())[paren_tuples_i]]
        paren_tuples_i += 1
    else:
        appendable += element

q_dict[last_kw] = appendable

q_dict

{'select': 'sum(quantity)', 'from': 'test_data'}

In [52]:

# Function to change keys from SQL to MQL
def change_key(q_dict, q_from, q_to, comp_df=df_comparison_ops):
    qd = q_dict.copy()

    def get_to_rep():
        return comp_df[comp_df[q_from] == key][q_to].item()

    def overwrite(q_dict, q_from, q_to, comp_df=df_comparison_ops):
        qd = q_dict.copy()
        to_rep = get_to_rep()
        qd[to_rep] = qd.pop(key)
        return qd

    for key in q_dict.keys():
        if isinstance(q_dict[key], dict):
            temp_qd = change_key(q_dict[key], q_from, q_to)
            try:
                qd[get_to_rep()] = temp_qd
            except:
                qd[key] = temp_qd

        elif isinstance(q_dict[key], list):
            temp_list = []
            for element in q_dict[key]:
                temp_list.append(change_key(element, q_from, q_to))
            qd.pop(key)
            qd[get_to_rep()] = temp_list

        elif key in list(comp_df[q_from]):
            qd = overwrite(qd, q_from, q_to, comp_df)
    return qd

# Convert the parsed SQL-like dictionary into MongoDB query syntax
mongo_query = change_key(q_dict, "SQL", "MQL")
print(mongo_query)

{'select': 'sum(quantity)', 'from': 'test_data'}


## Example query from SQL

In [27]:
query = """SELECT (a, SUM(b)) from (  \n x )  WHERE (a > 30    )"""

## Define the function to convert SQL into dictionary

In [65]:
class Sql2Dict():
    """
    This is a function-object that tries to convert
    a sql query into a string, into a dictionary object.
    """
    def __init__(self,query):
        if query:
            self.query = " ".join(query.lower().split())
            self.dict = {}
            self.parse_one_layer()
        else:
            raise Exception("query not found")

    def __repr__(self):
        """extracts the dictionary representation
        of this Sql2Dict object
        """
        return f"{self.dict}"

    def get_query(self):
        return self.query
    
    def get_dict(self):
        return self.dict.copy()
    
    
    # Function to parse parentheses in the query string
    def parse_paren(self,string):
        lvl = 0
        last_open = None
        parens_dict = {}
        for i, char in enumerate(string):
            assert lvl >= 0, "Check parenthesis"
            if char == "(":
                if lvl == 0:
                    last_open = i
                lvl += 1
            elif char == ")":
                lvl -= 1
                if lvl == 0:
                    parens_dict[(last_open, i+1)] = string[last_open+1:i]
        return parens_dict
    
    
    def parse_one_layer(self):
        query = f"{self.query}"
        # "normalise" the string
        q_list = query.lower().split()
        query = " ".join(q_list)
        
        # parse "inner" attributes out of the query 
        paren_tuples = self.parse_paren(query)
        for i in range(len(paren_tuples.keys())):
            i = (-i-1)
            vals = list(paren_tuples.keys())[i]
            query = (query[:vals[0]]+"{}"+query[vals[1]:])
        
        # set up to reconsistute the inner tuples
        q_dict = {}
        last_kw = False
        appendable = ""
        paren_tuples_i = 0
        
        # print(query,"\n")
        
        q_list = query.lower().split()
        
        for i,element in enumerate(q_list):
            # print(i,element)
            if (element in keywords):
                if i == 0:
                    # initialise "last_kw"
                    last_kw = element
                else:
                    q_dict[last_kw] = appendable
                    last_kw = element
                    appendable = ""
            elif (element =="{}"):
                # print(paren_tuples[list(paren_tuples.keys())[paren_tuples_i]])
                appendable += paren_tuples[list(paren_tuples.keys())[paren_tuples_i]]
                paren_tuples_i+=1
            else:
                appendable += element
        
        q_dict[last_kw] = appendable
        
        self.dict = q_dict
        return q_dict

## Here is the output looks like

In [55]:
query = """SELECT (a, SUM(b)) from (  \n x )  WHERE (a > 30    )"""

Sql2Dict(query).get_qeury()

'select (a, sum(b)) from ( x ) where (a > 30 )'

In [56]:
Sql2Dict(query).parse_one_layer()

{'select': 'a, sum(b)', 'from': ' x ', 'where': 'a > 30 '}

## Define Dictionary to Mongo query

In [38]:
class Dict2Mongo():
    """
    Tries to convert a "query_dict"
    to a MongoQuery String.
    """
    def __init__(self, query_dict: dict) -> None:
        self.query_dict = query_dict
        self.aggregations = self._parse_aggregations(query_dict.get('select', ''))

    def to_mongo_query(self):
        # Constructs the MongoDB aggregation pipeline
        pipeline = []
        for agg in self.aggregations:
            pipeline.append(agg.mongo_repr())
        return {'from': self.query_dict.get('from'), 'pipeline': pipeline}

    def _parse_aggregations(self, select_str):
        # Parses the select string to identify aggregation functions
        agg_funcs = []
        if 'avg' in select_str:
            agg_funcs.append(self.avg(select_str.split('avg(')[-1].split(')')[0]))
        if 'sum' in select_str:
            agg_funcs.append(self.sum(select_str.split('sum(')[-1].split(')')[0]))
        return agg_funcs

    class avg:
        def __init__(self, field):
            self.field = field
        
        def mongo_repr(self):
            return {'$group': {'_id': None, 'average': {'$avg': f'${self.field}'}}}

        def __repr__(self):
            return str(self.mongo_repr())

    class sum:
        def __init__(self, field):
            self.field = field
        
        def mongo_repr(self):
            return {'$group': {'_id': None, 'total': {'$sum': f'${self.field}'}}}

        def __repr__(self):
            return str(self.mongo_repr())

## Example of how the Dict2Mongo could be used to

In [69]:
# # Define a query for summing quantities in the Test_data collection
query = """select (sum(quantity)) from Test_data"""

dict_query = Sql2Dict(query).get_dict()
print(dict_query)

# query_1 = {
#     "select": "sum(quantity)",
#     "from": "Test_data"
# }

# Create an instance of Dict2Mongo with the updated query_1
dict2mongo = Dict2Mongo(dict_query)

# Generate and print the MongoDB query for the Test_data collection
mongo_query = dict2mongo.to_mongo_query()
print("MongoDB Query for Sum Aggregation in Test_data:", mongo_query)


{'select': 'sum(quantity)', 'from': 'test_data'}
MongoDB Query for Sum Aggregation in Test_data: {'from': 'test_data', 'pipeline': [{'$group': {'_id': None, 'total': {'$sum': '$quantity'}}}]}


In [18]:

# Connect to the local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# Select your database
db = client['Test_data']

# Use the generated aggregation pipeline from dict2mongo_1 or dict2mongo_2
pipeline = mongo_query['pipeline']  # For sum aggregation example

print(pipeline)

# Execute the aggregation query on the Test_data collection
result = db['Test_data'].aggregate(pipeline)

# Print the result
for doc in result:
    print(doc)


[{'$group': {'_id': None, 'total': {'$sum': '$quantity'}}}]
{'_id': None, 'total': 20}


# Consolidating the two-step process:

In [72]:
def ql_translator(query:str = """select (sum(quantity)) from Test_data"""):
    # use the given query for to construct a dictionary object
    dict_query = Sql2Dict(query).get_dict()
    print(dict_query)

    # Create an instance of Dict2Mongo with the updated query_1
    dict2mongo = Dict2Mongo(dict_query)

    # Generate and print the MongoDB query for the Test_data collection
    mongo_query = dict2mongo.to_mongo_query()
    print("MongoDB Query for Sum Aggregation in Test_data:", mongo_query)
    return mongo_query


In [73]:
ql_translator()

{'select': 'sum(quantity)', 'from': 'test_data'}
MongoDB Query for Sum Aggregation in Test_data: {'from': 'test_data', 'pipeline': [{'$group': {'_id': None, 'total': {'$sum': '$quantity'}}}]}


{'from': 'test_data',
 'pipeline': [{'$group': {'_id': None, 'total': {'$sum': '$quantity'}}}]}